In [ ]:
import os
import pandas as pd
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
from PIL import Image

# MÉTODO DE CAPTIONING 1 - VIT IMAGE CAPTIONING

In [ ]:
model_name = "nlpconnect/vit-gpt2-image-captioning"
model = VisionEncoderDecoderModel.from_pretrained(model_name)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_rang

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [ ]:
# Función para generar captions
def generate_caption(image_path):
    # Cargar la imagen
    image = Image.open(image_path).convert("RGB")
    # Preprocesar la imagen
    pixel_values = feature_extractor(images=[image], return_tensors="pt").pixel_values

    # Generar el caption
    output_ids = model.generate(pixel_values)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption

# Directorio donde se encuentran las imágenes
image_dir = "./data"

# Crear una lista para almacenar los resultados
results = []

# Recorrer los archivos en el directorio de imágenes
for idx, filename in enumerate(os.listdir(image_dir)):
    if filename.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif', 'tiff')):
        # Generar el caption para cada imagen
        image_path = os.path.join(image_dir, filename)
        caption = generate_caption(image_path)
        # Agregar el resultado al DataFrame
        results.append({"ID": os.path.splitext(filename)[0], "caption": caption})

# Crear un DataFrame con los resultados
df = pd.DataFrame(results)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [ ]:
df['ID'] = df['ID'].astype(int)
df = df.sort_values(by="ID").reset_index(drop=True)

In [ ]:
df

,ID,caption
0,1,a man in a car with a sign on his head
1,2,a dog with a funny face on it's face
2,3,a man in a suit and tie
3,4,a table with a picture of a cat and a toy frog
4,5,a dog sitting on a wooden post
...,...,...
994,995,a collage of photos of a soccer player
995,996,a man holding a remote control in front of a w...
996,997,a man in a hospital bed with a heart shaped sign
997,998,a man in a suit and tie standing next to a gro...


In [ ]:
df.to_csv('vlt-imagecaptioning.csv',index=False)

In [ ]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=06506607f805aaadba455aadf3d1eb677c7004771c0a6d90effa9ea11c10fe50
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [ ]:
df = pd.read_csv('vlt-imagecaptioning.csv')

In [ ]:
df.to_csv('vlt-imagecaptioning.csv')

In [ ]:
from googletrans import Translator

# Instanciar el traductor
translator = Translator()

# Función para traducir texto al español con manejo de errores
def translate_to_spanish(text):
    try:
        if text:  # Verificar que el texto no sea None o vacío
            translation = translator.translate(text, src='en', dest='es')
            return translation.text
        else:
            return ""  # Si el texto es None o vacío, devolver una cadena vacía
    except Exception as e:
        print(f"Error al traducir el texto: {text}. Error: {e}")
        return text  # Devolver el texto original si ocurre un error

# Traducir las captions al español directamente desde el DataFrame
df['caption_spanish'] = df['caption'].apply(translate_to_spanish)

Error al traducir el texto: a person on a snowboard . Error: 'NoneType' object is not iterable
Error al traducir el texto: a dog laying on a white surface . Error: 'Translator' object has no attribute 'raise_Exception'
Error al traducir el texto: a cartoon of a cartoon character with a computer . Error: 'Translator' object has no attribute 'raise_Exception'
Error al traducir el texto: a dog is standing on a beach with a map . Error: 'Translator' object has no attribute 'raise_Exception'
Error al traducir el texto: a cartoon of a cat with a heart shaped sign . Error: 'Translator' object has no attribute 'raise_Exception'
Error al traducir el texto: a baseball player is holding a bat and a ball . Error: 'NoneType' object is not iterable
Error al traducir el texto: a collage of photos of a cat and a dog . Error: 'Translator' object has no attribute 'raise_Exception'
Error al traducir el texto: a collage of different colored images of a person . Error: 'Translator' object has no attribute 

In [ ]:
df_new = df.drop(columns=['caption'])

In [ ]:
df_new.rename(columns={'caption_spanish': 'caption'}, inplace=True)

In [ ]:
df_new.to_csv('vlt-imagecaptioning_complete.csv',index=False,encoding='utf-8-sig')

In [ ]:
df.shape

(999, 3)

# MÉTODO DE CAPTIONING 2 - BLIP IMAGE CAPTIONING BASE

In [ ]:
#pip install transformers torch torchvision pillow

In [2]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import os
import pandas as pd

In [3]:
# Cargar el modelo BLIP y el procesador
model_name = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [4]:
# Función para generar captions con BLIP
def generate_blip_caption(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt")
        outputs = model.generate(**inputs)
        caption = processor.decode(outputs[0], skip_special_tokens=True)
        print(f"Procesada correctamente: {os.path.basename(image_path)}")
        return caption
    except Exception as e:
        print(f"Error procesando la imagen {os.path.basename(image_path)}: {e}")
        return "Error al generar caption"

# Directorio donde se encuentran las imágenes
image_dir = "./data"  # Cambia esto a la ruta donde están tus imágenes

# Crear un DataFrame para almacenar los resultados
results = []

# Recorrer las imágenes en la carpeta
for idx, filename in enumerate(os.listdir(image_dir)):
    if filename.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif', 'tiff')):
        image_path = os.path.join(image_dir, filename)
        caption = generate_blip_caption(image_path)
        results.append({"ID": os.path.splitext(filename)[0], "caption": caption})

# Crear un DataFrame con las captions
df = pd.DataFrame(results)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Procesada correctamente: 10.png
Procesada correctamente: 434.jpeg
Procesada correctamente: 440.jpeg
Procesada correctamente: 607.jpeg
Procesada correctamente: 90.jpeg
Procesada correctamente: 170.jpeg
Procesada correctamente: 648.jpeg
Procesada correctamente: 450.jpeg
Procesada correctamente: 75.jpeg
Procesada correctamente: 397.jpeg
Procesada correctamente: 197.jpeg
Procesada correctamente: 172.png
Procesada correctamente: 31.jpeg
Procesada correctamente: 727.jpeg
Procesada correctamente: 333.jpeg
Procesada correctamente: 126.jpeg
Procesada correctamente: 526.png
Procesada correctamente: 145.jpeg
Procesada correctamente: 866.jpeg
Procesada correctamente: 188.jpeg
Procesada correctamente: 108.jpeg
Procesada correctamente: 1.png
Procesada correctamente: 524.jpeg
Procesada correctamente: 449.png
Procesada correctamente: 395.jpeg
Procesada correctamente: 665.jpeg
Procesada correctamente: 144.jpeg
Procesada correctamente: 942.jpeg
Procesada correctamente: 551.jpeg
Procesada correctamente: 

In [5]:
df['ID'] = df['ID'].astype(int)
df = df.sort_values(by="ID").reset_index(drop=True)

In [6]:
df

,ID,caption
0,1,a man in a car with a quote on it
1,2,a dog with long blonde hair and a fake face
2,3,a man in a white robe with his hands up
3,4,mes de mes memes memes memes memes memes memes...
4,5,a small dog with a big smile on its face
...,...,...
994,995,a picture of a soccer team with the capt that ...
995,996,a man is dancing in the living room
996,997,a baby in a hospital bed with a white hat
997,998,a cartoon picture of a group of people


In [7]:
df.to_csv('blip-imagecaptioning.csv',index=False)

In [11]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=748b48cad901ea4ac964a329d969ff18f1ac83f6bbc970b63bf89e4ae5d49d2c
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succ

In [4]:
df = pd.read_csv('blip-imagecaptioning.csv')

In [6]:
from googletrans import Translator

# Instanciar el traductor
translator = Translator()

# Función para traducir texto al español con manejo de errores
def translate_to_spanish(text):
    try:
        if text:  # Verificar que el texto no sea None o vacío
            translation = translator.translate(text, src='en', dest='es')
            return translation.text
        else:
            return ""  # Si el texto es None o vacío, devolver una cadena vacía
    except Exception as e:
        print(f"Error al traducir el texto: {text}. Error: {e}")
        return text  # Devolver el texto original si ocurre un error

# Traducir las captions al español directamente desde el DataFrame
df['caption_spanish'] = df['caption'].apply(translate_to_spanish)

Error al traducir el texto: a picture of a soccer player with the words, que peo. Error: 'NoneType' object is not iterable
Error al traducir el texto: a hamy with the words, ' i ams sio '. Error: 'NoneType' object is not iterable
Error al traducir el texto: a baby is crying. Error: 'NoneType' object is not iterable
Error al traducir el texto: a red ball with a face and a capt that reads, ` ' ' ' ' ' '. Error: 'NoneType' object is not iterable
Error al traducir el texto: a crying smiley face with tears. Error: The read operation timed out
Error al traducir el texto: a screenshot of a baby in a sling. Error: 'NoneType' object is not iterable
Error al traducir el texto: a rada rada rada rada rada rada rada rada rada. Error: 'NoneType' object is not iterable
Error al traducir el texto: a green apple with a smiley face on it. Error: 'NoneType' object is not iterable
Error al traducir el texto: a poster with a picture of two graduates. Error: 'NoneType' object is not iterable
Error al traduc

In [7]:
df_new = df.drop(columns=['caption'])

In [8]:
df_new.rename(columns={'caption_spanish': 'caption'}, inplace=True)

In [9]:
df_new.to_csv('blip-imagecaptioning_complete.csv',index=False,encoding='utf-8-sig')